In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 97.2 MB/s eta 0:00:00


In [ ]:
import os
import pdfplumber
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
pdf_folder = '/content/drive/MyDrive/pdf'

In [ ]:
texts = []
names = []
phones = []
emails = []
labels = []
token_labels = []

In [ ]:
def extract_entities(text):
    names = re.findall(r"(\b[A-Z][a-z]+ [A-Z][a-z]+\b)", text)
    phones = re.findall(r"(\b\d{10}\b)", text)
    emails = re.findall(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", text)
    extracted_labels = ['O'] * len(text.split())
    for name in names:
        name_tokens = name.split()
        name_start = text.index(name_tokens[0])
        name_end = name_start + len(name_tokens)
        extracted_labels[name_start] = 'B-NAME'
        for i in range(name_start + 1, name_end):
            extracted_labels[i] = 'I-NAME'
    return extracted_labels

In [ ]:
for pdf_number in range(1, 201):
  pdf_filename = f"{pdf_number}.pdf"
  pdf_path = os.path.join(pdf_folder, pdf_filename)
  if os.path.exists(pdf_path):
    try:
      with pdfplumber.open(pdf_path) as pdf:
        pdf_text = ""
        for page in pdf.pages:
          pdf_text += page.extract_text()
        extracted_labels = extract_entities(pdf_text)
        texts.append(pdf_text)
        labels.append(extracted_labels)
    except Exception as e:
      continue

In [ ]:
def clean_text(text):
  text = text.replace("\n", " ").strip()
  return text

def preprocess_data(texts, labels, label_map):
  processed_texts = [clean_text(text) for text in texts]
  processed_labels = [[label_map[label] for label in doc_labels] for doc_labels in labels]
  return processed_texts, processed_labels

In [ ]:
label_map = {"O": 0, "B-NAME": 1, "I-NAME": 2, "B-PHONE": 3, "I-PHONE": 4, "B-EMAIL": 5, "I-EMAIL": 6}
preprocessed_texts, preprocessed_labels = preprocess_data(texts, labels, label_map)
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(preprocessed_texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding="post")
padded_labels = pad_sequences(preprocessed_labels, maxlen=max_sequence_length, padding="post")

In [ ]:
train_size = int(0.8 * len(padded_sequences))
x_train = padded_sequences[:train_size]
y_train = padded_labels[:train_size]
x_val = padded_sequences[train_size:]
y_val = padded_labels[train_size:]
embedding_dim = 128
hidden_units = 128
num_classes = len(label_map)

In [ ]:
print(len(x_train), len(y_train), len(x_val), len(y_val))

4 4 2 2


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_units, return_sequences=True)),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)

Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 1.9576 - accuracy: 0.0642 - val_loss: 1.9228 - val_accuracy: 0.8717
Epoch 2/10
1/1 [==============================] - 0s 313ms/step - loss: 1.8789 - accuracy: 0.9826 - val_loss: 1.9012 - val_accuracy: 0.9947
Epoch 3/10
1/1 [==============================] - 0s 427ms/step - loss: 1.7984 - accuracy: 1.0000 - val_loss: 1.8757 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 426ms/step - loss: 1.7066 - accuracy: 1.0000 - val_loss: 1.8432 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 307ms/step - loss: 1.5929 - accuracy: 1.0000 - val_loss: 1.7988 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 287ms/step - loss: 1.4423 - accuracy: 1.0000 - val_loss: 1.7339 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 334ms/step - loss: 1.2308 - accuracy: 1.0000 - val_loss: 1.6300 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=======

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
import openai

In [ ]:
with pdfplumber.open(pdf_path) as pdf:
  pdf_text = ""
  for page in pdf.pages:
    pdf_text += page.extract_text()
  preprocessed_pdf_text = clean_text(pdf_text)
  openai.api_base = "<api url>"
  openai.api_version = "<api version>"
  openai.api_key = "<api key>"
  response = openai.Completion.create(engine="code-davinci-002", prompt=f"Summarize:\n{preprocessed_pdf_text}", temperature=0.3, max_tokens=350, top_p=1, frequency_penalty=0, presence_penalty=0, stop=None)
  summary = response.choices[0].preprocessed_pdf_text.strip()
  print(summary)

InvalidRequestError: ignored